In [0]:
# define csv names
csv_names = ["payments", "riders", "stations", "trips"]

In [0]:
# read csv files into dataframes
for name in csv_names:
    globals()[f"df_{name}"] = spark.read.format("csv") \
        .option("inferSchema", "true") \
        .option("header", "false") \
        .option("sep", ",") \
        .load(f"/FileStore/data/{name}.csv")



In [0]:
# field names for source files

field_names = [
    ["payment_id",
    "date",
    "amount",
    "rider_id"],
    ["rider_id",
    "first",
    "last",
    "address",
    "birthday",
    "account_start_date",
    "account_end_date",
    "is_member"],
    ["station_id",
     "name",
     "latitude",
     "longitude"],
    ["trip_id",
     "rideable_type", 
     "started_at", 
     "ended_at", 
     "start_station_id", 
     "end_station_id",
     "rider_id"]    
]



In [0]:
# rename fields
for name, field in zip(csv_names, field_names):
    globals()[f"df_{name}"] = globals()[f"df_{name}"].toDF(*field)
    globals()[f"df_{name}"].createOrReplaceTempView(f"temp_{name}")


In [0]:
# save dataframes as delta using spark SQL
for name in csv_names:
    spark.sql(f"DROP TABLE IF EXISTS bronze_{name}")
    spark.sql(f"""
        CREATE OR REPLACE TABLE bronze_{name} 
        USING DELTA 
        LOCATION '/FileStore/delta/bronze/bronze_{name}'
        AS
        SELECT * FROM temp_{name}
    """)